In [19]:
import tweepy
import pandas as pd

def get_tweet_text(input_file, output_file, consumer_key, consumer_secret, access_token, access_token_secret):
    # Read the input CSV file
    df = pd.read_csv(input_file)

    # Authenticate with Twitter API
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    # Create a new column for tweet text
    df['Text'] = ''

    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        tweet_id = row['Tweet_ID']
        try:
            # Get the tweet object using the Twitter API
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            # Extract the tweet text
            tweet_text = tweet.full_text
            # Update the 'text' column in the dataframe
            df.at[index, 'Text'] = tweet_text
        except tweepy.errors.TweepyException as e:
            # Handle any errors that occur during API request
            print(f"Error retrieving tweet with ID {tweet_id}: {e}")

    # Save the updated dataframe to a new CSV file
    df.to_csv(output_file, index=False)

# Example usage
# input_file = 'input.csv'
# output_file = 'output.csv'
# consumer_key = 'your_consumer_key'
# consumer_secret = 'your_consumer_secret'
# access_token = 'your_access_token'
# access_token_secret = 'your_access_token_secret'

# get_tweet_text(input_file, output_file, consumer_key, consumer_secret, access_token, access_token_secret)

In [20]:
import os
import json

token_path = 'tokens.json'
if not os.path.isfile(token_path):
    raise Exception(f"{token_path} not found!")
with open(token_path) as f:
    # If you get an error here, it means your token is formatted incorrectly. Did you put it in quotes?
    tokens = json.load(f)
    consumer_key = tokens['TWITTER_AUTH']['consumer_key']
    consumer_secret = tokens['TWITTER_AUTH']['consumer_secret']
    access_token = tokens['TWITTER_AUTH']['access_token']
    access_token_secret = tokens['TWITTER_AUTH']['access_token_secret']

In [21]:
# Get the text versions of the tweets
input_file = 'Seed_MIWS/MIWS/MIWS.csv'
output_file = 'Seed_MIWS/MIWS/MIWS_with_text.csv'

# NOTE: For this to work, you need to have access to the GET /2/tweets or GET /2/tweets/:id endpoints
#       This requires a paid developer account and a project with elevated access
# get_tweet_text(input_file, output_file, consumer_key, consumer_secret, access_token, access_token_secret)

Error retrieving tweet with ID 1416578372508000257: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product
Error retrieving tweet with ID 1415105917268336640: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product
Error retrieving tweet with ID 1416069109494665216: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://devel

KeyboardInterrupt: 

In [10]:
# Examine the dataset
df = pd.read_csv(input_file, index_col='Unnamed: 0')

display(df.head())

,Tweet_ID,Created_Date,Geo_Enabled,Geographical_Location,Ideology,Labels
0,1416578372508000257,2021-07-18 01:59:37,False,Undefined,ISIS/Jihadist,Recruitment
1,1415105917268336640,2021-07-14 00:28:36,False,Undefined,ISIS/Jihadist,Radicalization
2,1416069109494665216,2021-07-16 16:15:59,False,"['Afghan', 'Pak', 'Pakistan']",ISIS/Jihadist,Radicalization
3,1415535727073443842,2021-07-15 04:56:31,True,"['USA', 'US']",ISIS/Jihadist,Propaganda
4,1415353483398729731,2021-07-14 16:52:21,False,Undefined,ISIS/Jihadist,Recruitment
